In [1]:
#Name: Srinitish Srinivasan
#Reg No: 21BAI1394

In [2]:
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor

In [3]:
dataset=FashionMNIST(root='data',train=True,download=True,transform=ToTensor())

100%|██████████| 26421880/26421880 [00:04<00:00, 6355854.48it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 169897.85it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:09<00:00, 489631.60it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 8480862.92it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
import torch.nn as nn
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))        
       
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))        

        x = x.view(x.size(0), -1)        
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)


In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()

In [6]:

k_folds = 5
batch_size = 64
kf = KFold(n_splits=k_folds, shuffle=True)

device=torch.device("mps")

for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    print(f"Fold {fold + 1}")
    print("-------")

    # Define the data loaders for the current fold
    train_loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        sampler=torch.utils.data.SubsetRandomSampler(train_idx),
    )
    test_loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        sampler=torch.utils.data.SubsetRandomSampler(test_idx),
    )

    # Initialize the model and optimizer
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

    # Train the model on the current fold
    for epoch in range(1, 11):
        train(model, device, train_loader, optimizer, epoch)

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.functional.nll_loss(output, target, reduction="sum").item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100.0 * correct / len(test_loader.dataset)

    # Print the results for the current fold
    print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n")

Fold 1
-------
Test set: Average loss: 0.2105, Accuracy: 6856/60000 (11.43%)

Fold 2
-------
Test set: Average loss: 0.1841, Accuracy: 7564/60000 (12.61%)

Fold 3
-------
Test set: Average loss: 0.1769, Accuracy: 8207/60000 (13.68%)

Fold 4
-------
Test set: Average loss: 0.1796, Accuracy: 7799/60000 (13.00%)

Fold 5
-------
Test set: Average loss: 0.2014, Accuracy: 7280/60000 (12.13%)

